In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import splitfolders

input_dir = './DATA/simpsons_dataset'
output_dir = './DATA/data/'

splitfolders.ratio(input_dir, output=output_dir, seed=1337, ratio=(.9, .1))

In [ ]:
train_dir = './DATA/data/train/'
val_dir = './DATA/data/val/'

In [ ]:
image_size = (64, 64, 3)
epochs = 50
batch_size = 128

labels = os.listdir(train_dir)

In [ ]:
# 이미지 증식
# shear_range : 반시계 방향으로 회전
# horizontal_flip : 좌우 방향으로 뒤집기
# rotation_range : 이미지 회전값
# width_shift_range : 좌우 이동
# height_shift_range : 상하 이동

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# 폴더에서 이미지를 가져와 제너레이터 객체로 정의
training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = image_size[:2],
                                                 batch_size=batch_size,
                                                 class_mode = 'sparse',
                                                 color_mode = 'rgb')

val_set = test_datagen.flow_from_directory(val_dir,
                                           target_size = image_size[:2],
                                           batch_size=batch_size,
                                           class_mode = 'sparse',
                                           color_mode = 'rgb')

In [ ]:
model = Sequential()
model.add(Input(shape=image_size))
model.add(Conv2D(32, (3, 3), padding = 'same', activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(labels), activation = 'softmax'))